In [5]:
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

from utilities import utils, process, evaluate, modify, plot
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [4]:
df = utils.load_images_to_dataframe('data/preprocessed')

# Encode the labels
label_encoder = LabelEncoder()
df['Target'] = label_encoder.fit_transform(df['Target'])

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

df.head(5)

,image,Target
filename,,
user072_abjadiyah_032.png,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",71
user002_qashtah_029.png,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",1
user003_mehras_043.png,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",2
user056_shateerah_035.png,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",55
user040_qashtah_027.png,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",39


In [6]:
# Custom Dataset class
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]  # Assuming the image path is in the first column
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx, 1]  # Assuming the label is in the second column

        if self.transform:
            image = self.transform(image)

        return image, label

In [1]:




# Define the transformation
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Create the dataset and dataloader
dataset = ImageDataset(dataframe=df, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 10)  # Assuming 10 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')

print('Finished Training')

RuntimeError: operator torchvision::nms does not exist

In [6]:
df['Target'].value_counts()

Target
71    100
1     100
2     100
55    100
39    100
     ... 
59    100
60    100
8     100
47     94
58     50
Name: count, Length: 82, dtype: int64